In [1]:
import pandas as pd
from pandas import Series, DataFrame

import numpy as np


import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import joblib

import tqdm

from sklearn.model_selection import train_test_split

from movies_utils.nnet_sim import *
from movies_utils.swa import SWA
from movies_utils.utils import *

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

seed_everything()

%matplotlib inline

In [2]:
DATA_FOLDER = 'movies_encoded_v1'
WINDOW_SIZE = 14
N_WIND_TO_TRAIN = 1
MIN_HIST_TO_TRAIN = 0
UNIQUE_MOVIES = False
MODEL_SIZE = 512
BATCH_SIZE = 64

# train on lb only
SWA_PATH = 'temp_swa_old_baseline_'
N_EP = 30
N_SWA = 5
SUB_NAME = 'old_nnet_baseline.csv'

In [3]:
data = pd.read_csv('movies/movies_dataset_10 months.csv')
data['ts'] = data['ts'].astype(np.datetime64)
data = data.sort_values('ts')

In [4]:
# create meta info 
movies = joblib.load(DATA_FOLDER + '/movies.pkl')
movies_meta = MetaInfo(3, '2020-04-01', ['genres', 'director', 'actors', 'writers', 'producer'], 
                       map_dup_targets = False)

movies_meta.fit(data, movies, 
               )


### VALIDATION PART

In [5]:
# split for local holdout validation
test = data[data['ts'].dt.month.isin([6,7])]
test = test.sort_values('ts')
data_train = data[~data['ts'].dt.month.isin([6,7])]

In [6]:
test = test.groupby('user_id')['movie_id'].agg(lambda x: x.tolist()[:5])

In [7]:
%%time
if N_WIND_TO_TRAIN > 1:
    augment = get_augmented_data(data_train, WINDOW_SIZE, N_WIND_TO_TRAIN)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.39 µs


In [8]:
# create train/valid/test data
data_train = data_train[data_train.groupby('user_id')['ts'].transform(lambda x: cut_hist(x, WINDOW_SIZE))]
valid_sl = data_train.groupby('user_id')['movie_id'].transform(get_split_flg)



In [9]:
valid = data_train[valid_sl].groupby('user_id')['movie_id'].agg(lambda x: x.tolist())
train = data_train[~valid_sl].groupby('user_id')['movie_id'].agg(lambda x: x.tolist())

train = train[train.map(lambda x: len(x) > MIN_HIST_TO_TRAIN)]

valid = valid[valid.index.isin(train.index)]

valid_features = train.loc[valid.index]
test_features = data_train.groupby('user_id')['movie_id'].agg(lambda x: x.tolist()).loc[test.index]

In [10]:
# add augment
if N_WIND_TO_TRAIN > 1:
    augment_train = augment.groupby('user_id')['movie_id'].agg(lambda x: x.tolist())
    augment_train = augment_train[augment_train.map(lambda x: len(x) > MIN_HIST_TO_TRAIN)]

    train.index = train.index.astype(str)
    train = pd.concat([train, augment_train], axis=0)

In [11]:
train_ds = TrainDataset(train, movies_meta, unique_feats=UNIQUE_MOVIES)
train_dl = DataLoader(train_ds, shuffle=True, batch_size = BATCH_SIZE, num_workers=10)

In [12]:
model = FFNet(movies_meta, MODEL_SIZE).cuda()
swa = SWA(10, SWA_PATH, rewrite=True)
scores = {}

In [13]:
model, swa, scores = train_loop(train_dl, 0,  unique_feats=UNIQUE_MOVIES, model=model, swa=swa, scores=scores,
               metric=map5,
               valid = (valid_features, valid), 
               test = (test_features, test)
)

In [ ]:
model = swa.set_weights(model, n=N_SWA)

In [21]:
recommender = Recommender(movies_meta, drop_dup_mov=True)

val_pred = predict_with_models(model, valid_features, recommender, unique_feats=UNIQUE_MOVIES)
val_score = map5(valid, val_pred)


test_pred = predict_with_models(model, test_features, recommender, unique_feats=UNIQUE_MOVIES)
test_score = map5(test, test_pred)

print(val_score, test_score)

0.0007884362680683313 0.0018007878446820486


#### TRAIN TO LB PART

In [24]:
ss = pd.read_csv('movies/submission.csv')

In [25]:
%%time
if N_WIND_TO_TRAIN > 1:
    augment = get_augmented_data(data, WINDOW_SIZE, N_WIND_TO_TRAIN)

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 10.3 µs


In [26]:
# train data
data_train = data[data.groupby('user_id')['ts'].transform(lambda x: cut_hist(x, WINDOW_SIZE))]
test_features = data_train.groupby('user_id')['movie_id'].agg(lambda x: x.tolist()).loc[ss['user_id'].values]

In [27]:
models = []


for i in range(42, 48):
    np.random.seed(i)
    print('State', i)
    # make split for new shuffle
    valid_sl = data_train.groupby('user_id')['movie_id'].transform(get_split_flg).values
    valid = data_train[valid_sl].groupby('user_id')['movie_id'].agg(lambda x: x.tolist())
    train = data_train[~valid_sl].groupby('user_id')['movie_id'].agg(lambda x: x.tolist())
    
    # cut short lists
    train = train[train.map(lambda x: len(x) > MIN_HIST_TO_TRAIN)]
    valid = valid[valid.index.isin(train.index)]
    
    valid_features = train.loc[valid.index]
    
    # augment
    if N_WIND_TO_TRAIN > 1:
        augment_train = augment.groupby('user_id')['movie_id'].agg(lambda x: x.tolist())
        augment_train = augment_train[augment_train.map(lambda x: len(x) > MIN_HIST_TO_TRAIN)]

        train.index = train.index.astype(str)
        train = pd.concat([train, augment_train], axis=0)
    
    # train model
    train_ds = TrainDataset(train, movies_meta, unique_feats=UNIQUE_MOVIES)
    train_dl = DataLoader(train_ds, shuffle=True, batch_size = BATCH_SIZE, num_workers=20)
    
    # 
    model, swa, scores = train_loop(train_dl, N_EP, embed_size=MODEL_SIZE, swa_path=SWA_PATH, 
                                    valid = (valid_features, valid))
    
    model = swa.set_weights(model, n=N_SWA)
    # 
    models.append(model)

State 42
Epoch  0



Score for valid = 0.015368783758614267
Epoch  1



Score for valid = 0.0132240640715217
Epoch  2



Score for valid = 0.015703420872912398
Epoch  3



Score for valid = 0.020041027710519237
Epoch  4


...


Score for valid = 0.05070829865484363
Epoch  26



Score for valid = 0.050812784202614716
Epoch  27



Score for valid = 0.0500647187087865
Epoch  28



Score for valid = 0.05206724826672675
Epoch  29



Score for valid = 0.051324501769417025


In [28]:
# make final submission - keep duplicate movies
recommender = Recommender(movies_meta, drop_dup_mov=False, )
test_pred = predict_with_models(models, test_features, recommender, unique_feats=UNIQUE_MOVIES)
ss['movie_id'] = [' '.join(map(str, x)) for x in test_pred]
ss.to_csv('{0}_dd.csv'.format(SUB_NAME), index=False)